### [02] 필요한 모듈과 클래스 불러오기

In [ ]:
from openpyxl import load_workbook # 엑셀 파일 작업
from pptx import Presentation # 파워포인트 파일 작업
from pptx.util import Inches # 객체를 인치로 지정
from pptx.enum.chart import XL_CHART_TYPE # 차트 유형
from pptx.chart.data import CategoryChartData # 차트 데이터 추가 클래스

### [03] 엑셀 데이터 불러온 다음 파워포인트 객체 생성하기

In [ ]:
# ➊ 엑셀 파일에서 데이터 불러오기
wb = load_workbook('제품별 판매내역.xlsx')
sheet_names = wb.sheetnames

# ➋ 파워포인트 객체 생성
prs = Presentation()

# ➌ 표지 슬라이드 추가
slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '상반기 제품별 판매현황 분석'

# prs.save('상반기_제품별_판매현황_분석_작성중.pptx') # <- 중간 파일을 확인하고 싶다면 여기 주석을 해제하고 실행하세요

### [04] 판매량 데이터 가져오기

In [ ]:
wb = load_workbook('제품별 판매내역.xlsx')
sheet_names = wb.sheetnames

prs = Presentation()

slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '상반기 제품별 판매현황 분석'

# ➊ 제품별 판매량 데이터를 저장하기 위한 딕셔너리
sales_data = {}

# ➋ 엑셀 전체 시트를 순회
for sheet_name in sheet_names:
  sheet = wb[sheet_name]

  # ➌ 현재 시트의 전체 데이터 추출하기
  data = [list(row) for row in sheet.iter_rows(values_only=True)]
  print(sheet_name, '전체 데이터:', data)

  # ➍ 제품의 '판매량' 열 데이터 추출해서 sales_data에 추가하기
  sales_data[sheet_name] = [row[1] for row in data[1:]]
  print('월별 판매량:', sales_data)
  print('-----------------------------')

### [05] 제품별 슬라이드 추가하고 표 생성하기

In [ ]:
wb = load_workbook('제품별 판매내역.xlsx')
sheet_names = wb.sheetnames

prs = Presentation()

slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '상반기 제품별 판매현황 분석'

sales_data = {}

for sheet_name in sheet_names:
  sheet = wb[sheet_name]

  data = [list(row) for row in sheet.iter_rows(values_only=True)]
  print(sheet_name, '전체 데이터:', data)

  sales_data[sheet_name] = [row[1] for row in data[1:]]
  print('월별 판매량:', sales_data)
  print('-----------------------------')
    
  # ➊ 슬라이드 추가
  slide_layout = prs.slide_layouts[5]
  slide = prs.slides.add_slide(slide_layout)
  slide.shapes.title.text = f'{sheet_name}'
  # ➋ 표 추가
  rows, cols = len(data), len(data[0])
  table = slide.shapes.add_table(rows+1, cols, Inches(0.3), Inches(2), Inches(4.3), Inches(3.8)).table
  # ➌ '월별 판매 현황' 추가
  table.cell(0, 0).text = '월별 판매 현황'
  table.cell(0, 0).merge(table.cell(0, cols - 1))
  # ➍ 데이터를 표에 추가
  for row_idx, row_val in enumerate(data):
    for col_idx, val in enumerate(row_val):
      table.cell(row_idx+1, col_idx).text = str(val)

# prs.save('상반기_제품별_판매현황_분석_작성중.pptx') <- 중간 파일을 확인하고 싶다면 여기 주석을 해제하고 실행하세요

### [06] 제품별 슬라이드에 차트 추가하기

In [ ]:
wb = load_workbook('제품별 판매내역.xlsx')
sheet_names = wb.sheetnames

prs = Presentation()

slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '상반기 제품별 판매현황 분석'

sales_data = {}

for sheet_name in sheet_names:
  sheet = wb[sheet_name]

  data = [list(row) for row in sheet.iter_rows(values_only=True)]
  print(sheet_name, '전체 데이터:', data)

  sales_data[sheet_name] = [row[1] for row in data[1:]]
  print('월별 판매량:', sales_data)
  print('-----------------------------')
    
  slide_layout = prs.slide_layouts[5]
  slide = prs.slides.add_slide(slide_layout)
  slide.shapes.title.text = f'{sheet_name}'

  rows, cols = len(data), len(data[0])
  table = slide.shapes.add_table(rows+1, cols, Inches(0.3), Inches(2), Inches(4.3), Inches(3.8)).table

  table.cell(0, 0).text = '월별 판매 현황'
  table.cell(0, 0).merge(table.cell(0, cols - 1))

  for row_idx, row_val in enumerate(data):
    for col_idx, val in enumerate(row_val):
      table.cell(row_idx+1, col_idx).text = str(val)
        
  # ➊ 차트 데이터 설정
  chart_data = CategoryChartData()
  chart_data.categories = [row[0] for row in data[1:]]
  chart_data.add_series('상반기 판매량 변화', [row[1] for row in data[1:]]) 
  # ➋ 차트 추가
  x, y, cx, cy = Inches(5), Inches(1.8), Inches(4.5), Inches(4.2)
  chart = slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data).chart 
  # ➌ 차트 설정
  chart.has_legend = False
  chart.plots[0].has_data_labels = True

# prs.save('상반기_제품별_판매현황_분석_작성중.pptx') <- 중간 파일을 확인하고 싶다면 여기 주석을 해제하고 실행하세요

### [07] 전체 판매 추이 슬라이드에 선 차트 추가하기

In [ ]:
wb = load_workbook('제품별 판매내역.xlsx')
sheet_names = wb.sheetnames

prs = Presentation()

slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '상반기 제품별 판매현황 분석'

sales_data = {}

for sheet_name in sheet_names:
  sheet = wb[sheet_name]

  data = [list(row) for row in sheet.iter_rows(values_only=True)]
  print(sheet_name, '전체 데이터:', data)

  sales_data[sheet_name] = [row[1] for row in data[1:]]
  print('월별 판매량:', sales_data)
  print('-----------------------------')
    
  slide_layout = prs.slide_layouts[5]
  slide = prs.slides.add_slide(slide_layout)
  slide.shapes.title.text = f'{sheet_name}'

  rows, cols = len(data), len(data[0])
  table = slide.shapes.add_table(rows+1, cols, Inches(0.3), Inches(2), Inches(4.3), Inches(3.8)).table

  table.cell(0, 0).text = '월별 판매 현황'
  table.cell(0, 0).merge(table.cell(0, cols - 1))

  for row_idx, row_val in enumerate(data):
    for col_idx, val in enumerate(row_val):
      table.cell(row_idx+1, col_idx).text = str(val)
        
  chart_data = CategoryChartData()
  chart_data.categories = [row[0] for row in data[1:]]
  chart_data.add_series('상반기 판매량 변화', [row[1] for row in data[1:]]) 

  x, y, cx, cy = Inches(5), Inches(1.8), Inches(4.5), Inches(4.2)
  chart = slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data).chart 

  chart.has_legend = False
  chart.plots[0].has_data_labels = True


# ➊ 전체 판매 추이 슬라이드 생성
slide_layout = prs.slide_layouts[5]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '제품별 판매 추이'

# ➋ 전체 판매 추이 차트 데이터 설정
chart_data = CategoryChartData()
chart_data.categories = [row[0] for row in data[1:]]
for sheet_name, sales in sales_data.items():
  chart_data.add_series(sheet_name, sales)

# ➌ 전체 판매 추이 차트 추가
x, y, cx, cy = Inches(1), Inches(2), Inches(8), Inches(4.5)
chart = slide.shapes.add_chart(XL_CHART_TYPE.LINE, x, y, cx, cy, chart_data).chart

# ➍ 차트 설정
chart.has_legend = True
chart.legend.include_in_layout = False
chart.category_axis.has_major_gridlines = False

# prs.save('상반기_제품별_판매현황_분석_작성중.pptx') <- 중간 파일을 확인하고 싶다면 여기 주석을 해제하고 실행하세요

### [08] 마무리 슬라이드 추가한 다음 파일 저장하기

In [ ]:
wb = load_workbook('제품별 판매내역.xlsx')
sheet_names = wb.sheetnames

prs = Presentation()

slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '상반기 제품별 판매현황 분석'

sales_data = {}

for sheet_name in sheet_names:
  sheet = wb[sheet_name]

  data = [list(row) for row in sheet.iter_rows(values_only=True)]
  print(sheet_name, '전체 데이터:', data)

  sales_data[sheet_name] = [row[1] for row in data[1:]]
  print('월별 판매량:', sales_data)
  print('-----------------------------')
    
  slide_layout = prs.slide_layouts[5]
  slide = prs.slides.add_slide(slide_layout)
  slide.shapes.title.text = f'{sheet_name}'

  rows, cols = len(data), len(data[0])
  table = slide.shapes.add_table(rows+1, cols, Inches(0.3), Inches(2), Inches(4.3), Inches(3.8)).table

  table.cell(0, 0).text = '월별 판매 현황'
  table.cell(0, 0).merge(table.cell(0, cols - 1))

  for row_idx, row_val in enumerate(data):
    for col_idx, val in enumerate(row_val):
      table.cell(row_idx+1, col_idx).text = str(val)
        
  chart_data = CategoryChartData()
  chart_data.categories = [row[0] for row in data[1:]]
  chart_data.add_series('상반기 판매량 변화', [row[1] for row in data[1:]]) 

  x, y, cx, cy = Inches(5), Inches(1.8), Inches(4.5), Inches(4.2)
  chart = slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data).chart 

  chart.has_legend = False
  chart.plots[0].has_data_labels = True

slide_layout = prs.slide_layouts[5]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '제품별 판매 추이'

chart_data = CategoryChartData()
chart_data.categories = [row[0] for row in data[1:]]
for sheet_name, sales in sales_data.items():
  chart_data.add_series(sheet_name, sales)

x, y, cx, cy = Inches(1), Inches(2), Inches(8), Inches(4.5)
chart = slide.shapes.add_chart(XL_CHART_TYPE.LINE, x, y, cx, cy, chart_data).chart

chart.has_legend = True
chart.legend.include_in_layout = False
chart.category_axis.has_major_gridlines = False

# ➊ 마지막 슬라이드 추가
slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '감사합니다.'

# ➋ 결과 저장
prs.save('상반기_제품별_판매현황_분석.pptx')

### [전체 코드]

In [ ]:
from openpyxl import load_workbook
from pptx import Presentation
from pptx.util import Inches
from pptx.enum.chart import XL_CHART_TYPE
from pptx.chart.data import CategoryChartData

wb = load_workbook('제품별 판매내역.xlsx')
sheet_names = wb.sheetnames

# 프레젠테이션 추가 + 표지 레이아웃 추가
prs = Presentation()
slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '상반기 제품별 판매현황 분석'

# 제품별 판매량 데이터를 저장하기 위한 딕셔너리
sales_data = {}

# 엑셀 전체 시트를 순회
for sheet_name in sheet_names:
  sheet = wb[sheet_name]
    
  # 현재 시트의 전체 데이터 추출하기
  data = [list(row) for row in sheet.iter_rows(values_only=True)]
  print(sheet_name, '전체 데이터:', data)

  # 제품의 '판매량' 열 데이터 추출해서 sales_data에 추가하기
  sales_data[sheet_name] = [row[1] for row in data[1:]]
  print('월별 판매량:', sales_data)
  print('-----------------------------')
    
  # 슬라이드 추가
  slide_layout = prs.slide_layouts[5]
  slide = prs.slides.add_slide(slide_layout)
  slide.shapes.title.text = f'{sheet_name}'
    
  # 표 추가
  rows, cols = len(data), len(data[0])
  table = slide.shapes.add_table(rows+1, cols, Inches(0.3), Inches(2), Inches(4.3), Inches(3.8)).table
    
  # '월별 판매 현황' 추가
  table.cell(0, 0).text = '월별 판매 현황'
  table.cell(0, 0).merge(table.cell(0, cols - 1))
    
  # 데이터 표에 추가
  for row_idx, row_val in enumerate(data):
    for col_idx, val in enumerate(row_val):
      table.cell(row_idx+1, col_idx).text = str(val)
        
  # 차트 데이터 설정
  chart_data = CategoryChartData()
  chart_data.categories = [row[0] for row in data[1:]]
  chart_data.add_series('상반기 판매량 변화', [row[1] for row in data[1:]])
    
  # 차트 추가
  x, y, cx, cy = Inches(5), Inches(1.8), Inches(4.5), Inches(4.2)
  chart = slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data).chart
    
  # 차트 설정
  chart.has_legend = False
  chart.plots[0].has_data_labels = True

# 전체 판매 추이 슬라이드 생성
slide_layout = prs.slide_layouts[5]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '제품별 판매 추이'

# 전체 판매 추이 차트 데이터 설정
chart_data = CategoryChartData()
chart_data.categories = [row[0] for row in data[1:]]
for sheet_name, sales in sales_data.items():
  chart_data.add_series(sheet_name, sales)

# 전체 판매 추이 차트 추가
x, y, cx, cy = Inches(1), Inches(2), Inches(8), Inches(4.5)
chart = slide.shapes.add_chart(XL_CHART_TYPE.LINE, x, y, cx, cy, chart_data).chart

# 차트 설정
chart.has_legend = True
chart.legend.include_in_layout = False
chart.category_axis.has_major_gridlines = False

# 마지막 슬라이드 추가
slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = '감사합니다.'

# PPT 파일 저장
prs.save('상반기_제품별_판매현황_분석.pptx')